<a href="https://colab.research.google.com/github/brs1977/neural-university/blob/master/HW/01_mnist_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задание Light**

Проверьте, как сеть распознает разные цифры из тестового набора данных 
Попробуйте сами написать цифру и распознать ее с помощью сеть 
В цикле распознайте все цифры тестовой выборки и посчитайте точность распознавания на тестовой 
Выберите 10 разных вариантов количества слоёв и нейронов в скрытых слоях от небольшого до большого и заполните таблицу с точностью обучения - на обучающей и на тестовой выборке 
Напишите ваши выводы - как влияет количество нейронов в сети на точность на тестовой выборке

**Задание Pro**

Запустите в цикле поштучное распознавание всех цифр обучающей выборки 
Выделите те, на которых была ошибка и точность распознавания была ниже некоторого порога alpha (например, 0.8) 
Выбросите все ошибочные цифры из обучающей выборки Переобучите сеть на «очищенной» выборке 
Посчитайте точность на тестовой выборке и сравните с точность на тестовой до очистки обучающей и коэффициент роста точности Составьте таблицу коэффициентов роста точности на 10 примерах в зависимости от размера сети и от разных alpha 
Напишите ваши выводы - как влияет очистка данных на точность на тестовой выборке

In [0]:
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten 
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import tensorflow as tf
from sklearn.model_selection import train_test_split
from google.colab import files
import numpy as np
import random
import os
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

##Загрузка и подготовка данных

In [2]:
(x_train_org, y_train_org), (x_test_org, y_test_org) = mnist.load_data()

x_train = x_train_org.reshape(60000, 784)
x_test = x_test_org.reshape(10000, 784)

x_train = x_train.astype('float32') / 255
y_train = utils.to_categorical(y_train_org, 10)

#x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

x_test = x_test.astype('float32') / 255
y_test = utils.to_categorical(y_test_org, 10)

y_train[100:110]

11493376/11490434 [==============================] - 0s 0us/step


array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

#Функции для работы Keras


In [0]:
# модель НС
def create_model(layers, TPU=False):
  model = Sequential()
  # Входной полносвязный слой, 800 нейронов, 784 входа в каждый нейрон
  #model.add(Dense(800, input_dim=784, activation="relu"))
  for layer in layers:
    model.add(layer)
  
  # Выходной полносвязный слой, 10 нейронов (по количеству рукописных цифр)
  model.add(Dense(10, activation="softmax"))

  if TPU:
    model = tf.contrib.tpu.keras_to_tpu_model(
        model,
        strategy=tf.contrib.tpu.TPUDistributionStrategy(
            tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
        )
    )  
  
  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  return model


def callbacks(verbose=0):
  #будем сохранять последнюю лучшую модель 
  сheckpoint = ModelCheckpoint('mnist-cnn.hdf5', 
                                monitor='acc', 
                                save_best_only=True,
                                verbose=verbose)

  #уменьшаем скорость обучения в 2 раза, если за 3 эпохи, метрика не улучшилась
  learning_rate_reduction = ReduceLROnPlateau(monitor='acc', 
                                              patience=3, 
                                              verbose=verbose, 
                                              factor=0.5, 
                                              min_lr=0.00001)

  #остановим обучение если за 4 эпох, метрика не улучшилась
  earlystop = EarlyStopping(monitor='acc', min_delta=0.001, patience=4,verbose=verbose, mode='auto')  
  return [сheckpoint,learning_rate_reduction,earlystop]

#загружаем сохраненную best model, делаем предсказание на test  
#return (train_acc,test_acc)
def predict(layers,verbose=0):
  model = create_model(layers)
  history = model.fit(x_train, y_train, batch_size=batch_size, epochs=100, callbacks=callbacks(),verbose=verbose)  
  train_acc = np.max(history.history['acc'])
  
  model = create_model(layers)
  model.load_weights('mnist-cnn.hdf5')
  score = model.evaluate(x_test, y_test, verbose=verbose)
  
  return [train_acc,score[1]]
  #predictions = model.predict(x_test)
  #train_acc = np.max(history.history['acc'])
  #test_acc = accuracy_score(np.argmax(y_test,axis=1), np.argmax(predictions,axis=1))
  #return (train_acc,test_acc)
  
#создает и тестирует модели (глубиной 0:2) с одинаковым количеством нейронов   
#return [кол слоев, кол нейронов в слое, train_acc,test_acc]
def test_layers(units=800,batch_size = 256, range_len = 10, verbose = 0):  
  res = []
  
  train_acc = 0
  test_acc = 0
  for i in range(range_len):
    trn_acc,tst_acc = predict([Dense(units, input_dim=784, activation="relu")])
    train_acc += trn_acc/range_len
    test_acc += tst_acc/range_len
    
  res.append([0,units,train_acc,test_acc])
                        
  train_acc = 0
  test_acc = 0
  for i in range(range_len):
    trn_acc,tst_acc = predict([Dense(units, input_dim=784, activation="relu"),
                                  Dense(units, activation="relu")])
    train_acc += trn_acc/range_len
    test_acc += tst_acc/range_len

  res.append([1,units,train_acc,test_acc])

  train_acc = 0
  test_acc = 0
  for i in range(range_len):
    trn_acc,tst_acc = predict([Dense(units, input_dim=784, activation="relu"),
                                   Dense(units, activation="relu"),
                                   Dense(units, activation="relu")])
    train_acc += trn_acc/range_len
    test_acc += tst_acc/range_len
    
  res.append([2,units,train_acc,test_acc])
  
  return res     

#создает и тестирует модели (глубиной 3:5) с уменьшающимся количеством нейронов (через 100) в глубину НС  
#return [кол слоев, кол нейронов в 1 слое, train_acc,test_acc]
def test_deep_layers(units=800,batch_size = 256, range_len = 10, verbose = 0):  
  res = []
  
  train_acc = 0
  test_acc = 0
  for i in range(range_len):
    trn_acc,tst_acc = predict([Dense(units, input_dim=784, activation="relu"),
                                   Dense(units-100, activation="relu"),
                                   Dense(units-200, activation="relu"),
                                   Dense(units-300, activation="relu")])
    train_acc += trn_acc/range_len
    test_acc += tst_acc/range_len
    
  res.append([4,units,train_acc,test_acc])

  train_acc = 0
  test_acc = 0
  for i in range(range_len):
    trn_acc,tst_acc = predict([Dense(units, input_dim=784, activation="relu"),
                                   Dense(units-100, activation="relu"),
                                   Dense(units-200, activation="relu"),
                                   Dense(units-300, activation="relu"),
                                   Dense(units-400, activation="relu")])
    train_acc += trn_acc/range_len
    test_acc += tst_acc/range_len
    
  res.append([5,units,train_acc,test_acc])
  
  train_acc = 0
  test_acc = 0
  for i in range(range_len):
    trn_acc,tst_acc = predict([Dense(units, input_dim=784, activation="relu"),
                                   Dense(units-100, activation="relu"),
                                   Dense(units-200, activation="relu"),
                                   Dense(units-300, activation="relu"),
                                   Dense(units-400, activation="relu"),
                                   Dense(units-500, activation="relu")])
    train_acc += trn_acc/range_len
    test_acc += tst_acc/range_len
   
  res.append([6,units,train_acc,test_acc])
    
  return res 

###проверка модели с 0:2 скрытыми слоями количеством нейронов 400:1100 с шагом 100

In [4]:
%%time 
batch_size = 256
range_len = 3

res = []

random.seed(666)
res += test_layers(units=400,batch_size=batch_size,range_len=range_len)
print(res[len(res)-3:len(res)])

random.seed(666)
res += test_layers(units=500,batch_size=batch_size,range_len=range_len)
print(res[len(res)-3:len(res)])

random.seed(666)
res += test_layers(units=600,batch_size=batch_size,range_len=range_len)
print(res[len(res)-3:len(res)])

random.seed(666)
res += test_layers(units=700,batch_size=batch_size,range_len=range_len)
print(res[len(res)-3:len(res)])

random.seed(666)
res += test_layers(units=800,batch_size=batch_size,range_len=range_len)
print(res[len(res)-3:len(res)])

random.seed(666)
res += test_layers(units=900,batch_size=batch_size,range_len=range_len)
print(res[len(res)-3:len(res)])

random.seed(666)
res += test_layers(units=1000,batch_size=batch_size,range_len=range_len)
print(res[len(res)-3:len(res)])

random.seed(666)
res += test_layers(units=1100,batch_size=batch_size,range_len=range_len)
print(res[len(res)-3:len(res)])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
[[0, 400, 0.9997999866803486, 0.9811999996503196], [1, 400, 0.9994777838389077, 0.9839666485786438], [2, 400, 1.0, 0.9853999813397725]]
[[0, 500, 0.9998944600423179, 0.9819333354632058], [1, 500, 0.9991111159324646, 0.9839333494504292], [2, 500, 0.9983333349227905, 0.982366681098938]]
[[0, 600, 0.9998722275098164, 0.9824000000953674], [1, 600, 0.9978777766227722, 0.9800999959309897], [2, 600, 0.9966166615486145, 0.9808666507403057]]
[[0, 700, 0.999916672706604, 0.9828000068664551], [1, 700, 0.9988222320874531, 0.9841333230336506], [2, 700, 0.9985611041386921, 0.9843999942143757]]
[[0, 800, 0.9999277790387471, 0.9831666747728984], [1, 800, 0.9983111023902893, 0.9829666813214619], [2, 800, 0.9992611209551492, 0.9846333265304565]]
[[0, 900, 0.999911109606425, 0.9835333426793416], [1, 900, 0.9989555478096008, 0.9841666618982952], [2, 900, 0.9984999895095825, 0.98436665534

###результаты модели на тестовых данных, сортировка по уменьшению точности 

In [7]:
res = [[0, 400, 0.9997999866803486, 0.9811999996503196], [1, 400, 0.9994777838389077, 0.9839666485786438], [2, 400, 1.0, 0.9853999813397725]] + \
  [[0, 500, 0.9998944600423179, 0.9819333354632058], [1, 500, 0.9991111159324646, 0.9839333494504292], [2, 500, 0.9983333349227905, 0.982366681098938]] + \
  [[0, 600, 0.9998722275098164, 0.9824000000953674], [1, 600, 0.9978777766227722, 0.9800999959309897], [2, 600, 0.9966166615486145, 0.9808666507403057]] + \
  [[0, 700, 0.999916672706604, 0.9828000068664551], [1, 700, 0.9988222320874531, 0.9841333230336506], [2, 700, 0.9985611041386921, 0.9843999942143757]] + \
  [[0, 800, 0.9999277790387471, 0.9831666747728984], [1, 800, 0.9983111023902893, 0.9829666813214619], [2, 800, 0.9992611209551492, 0.9846333265304565]] + \
  [[0, 900, 0.999911109606425, 0.9835333426793416], [1, 900, 0.9989555478096008, 0.9841666618982952], [2, 900, 0.9984999895095825, 0.9843666553497314]] + \
  [[0, 1000, 0.9998333255449932, 0.9830000003178916], [1, 1000, 0.9978777766227722, 0.9831333359082541], [2, 1000, 0.9984611074129739, 0.9837999939918518]] + \
  [[0, 1100, 0.9999610980351765, 0.9834333260854085], [1, 1100, 0.9985666672388711, 0.9842999974886577], [2, 1100, 0.9987611174583435, 0.9853333234786987]]

df = pd.DataFrame(res,columns=('level','units','train_acc','test_acc'))
#df[df['level']==1].sort_values('test_acc', ascending=False)
df.sort_values('test_acc', ascending=False)

,level,units,train_acc,test_acc
2,2,400,1.000000,0.985400
23,2,1100,0.998761,0.985333
14,2,800,0.999261,0.984633
11,2,700,0.998561,0.984400
17,2,900,0.998500,0.984367
22,1,1100,0.998567,0.984300
16,1,900,0.998956,0.984167
10,1,700,0.998822,0.984133
1,1,400,0.999478,0.983967
4,1,500,0.999111,0.983933


###результаты модели на обучающих данных, сортировка по уменьшению точности

In [8]:
df.sort_values('train_acc', ascending=False)

,level,units,train_acc,test_acc
2,2,400,1.000000,0.985400
21,0,1100,0.999961,0.983433
12,0,800,0.999928,0.983167
9,0,700,0.999917,0.982800
15,0,900,0.999911,0.983533
3,0,500,0.999894,0.981933
6,0,600,0.999872,0.982400
18,0,1000,0.999833,0.983000
0,0,400,0.999800,0.981200
1,1,400,0.999478,0.983967


###проверка модели с 3:5  скрытыми слоями с уменьшающимся количеством нейронов 600:1000 с шагом 100 в глубину НС 

In [8]:
%%time 

range_len = 3
batch_size = 256
res1 = []

random.seed(666)
res1 += test_deep_layers(units=600,batch_size=batch_size,range_len=range_len )
print(res1[len(res1)-3:len(res1)])

random.seed(666)
res1 += test_deep_layers(units=700,batch_size=batch_size,range_len=range_len )
print(res1[len(res1)-3:len(res1)])

random.seed(666)
res1 += test_deep_layers(units=800,batch_size=batch_size,range_len=range_len )
print(res1[len(res1)-3:len(res1)])

random.seed(666)
res1 += test_deep_layers(units=900,batch_size=batch_size,range_len=range_len )
print(res1[len(res1)-3:len(res1)])

random.seed(666)
res1 += test_deep_layers(units=1000,batch_size=batch_size,range_len=range_len )
print(res1[len(res1)-3:len(res1)])

[[4, 600, 0.9963166515032449, 0.9809666673342388], [5, 600, 0.9973222215970357, 0.9823333422342935], [6, 600, 0.9977777798970541, 0.9818333188692729]]
[[4, 700, 0.9976277748743694, 0.9826666712760925], [5, 700, 0.9974944392840068, 0.9823000033696493], [6, 700, 0.996572236220042, 0.9810666839281716]]
[[4, 800, 0.9976388812065125, 0.9839333494504294], [5, 800, 0.9975888927777607, 0.9829000035921733], [6, 800, 0.997349997361501, 0.9826000134150188]]
[[4, 900, 0.996161123116811, 0.9816666642824807], [5, 900, 0.9986166755358377, 0.9842999974886575], [6, 900, 0.9976666569709778, 0.982699990272522]]
[[4, 1000, 0.9989555478096008, 0.9841333230336509], [5, 1000, 0.9976611137390137, 0.9814333319664001], [6, 1000, 0.9980833331743875, 0.985099991162618]]
CPU times: user 2h 6min 36s, sys: 8min 46s, total: 2h 15min 22s
Wall time: 2h 9min 51s


###результаты модели на тестовых данных, сортировка по уменьшению точности 

In [5]:
res1 = [[4, 600, 0.9963166515032449, 0.9809666673342388], [5, 600, 0.9973222215970357, 0.9823333422342935], [6, 600, 0.9977777798970541, 0.9818333188692729]] + \
    [[4, 700, 0.9976277748743694, 0.9826666712760925], [5, 700, 0.9974944392840068, 0.9823000033696493], [6, 700, 0.996572236220042, 0.9810666839281716]] + \
    [[4, 800, 0.9976388812065125, 0.9839333494504294], [5, 800, 0.9975888927777607, 0.9829000035921733], [6, 800, 0.997349997361501, 0.9826000134150188]] + \
    [[4, 900, 0.996161123116811, 0.9816666642824807], [5, 900, 0.9986166755358377, 0.9842999974886575], [6, 900, 0.9976666569709778, 0.982699990272522]] + \
    [[4, 1000, 0.9989555478096008, 0.9841333230336509], [5, 1000, 0.9976611137390137, 0.9814333319664001], [6, 1000, 0.9980833331743875, 0.985099991162618]]

df1 = pd.DataFrame(res1,columns=('level','units','train_acc','test_acc'))
#df1[df1['level']==1].sort_values('test_acc', ascending=False)
df1.sort_values('test_acc', ascending=False)

,level,units,train_acc,test_acc
14,6,1000,0.998083,0.985100
10,5,900,0.998617,0.984300
12,4,1000,0.998956,0.984133
6,4,800,0.997639,0.983933
7,5,800,0.997589,0.982900
11,6,900,0.997667,0.982700
3,4,700,0.997628,0.982667
8,6,800,0.997350,0.982600
1,5,600,0.997322,0.982333
4,5,700,0.997494,0.982300


###результаты модели на тренировочных данных, сортировка по уменьшению точности

In [6]:
df1.sort_values('train_acc', ascending=False)

,level,units,train_acc,test_acc
12,4,1000,0.998956,0.984133
10,5,900,0.998617,0.984300
14,6,1000,0.998083,0.985100
2,6,600,0.997778,0.981833
11,6,900,0.997667,0.982700
13,5,1000,0.997661,0.981433
6,4,800,0.997639,0.983933
3,4,700,0.997628,0.982667
7,5,800,0.997589,0.982900
4,5,700,0.997494,0.982300


##Вывод Lite

проведено тестирование 2 типов моделей с 0:2 скрытыми слоями и однаковым кол-вом нейронов 400:1100 шагом 100 и глубокой модели 3:5 скрытыми слоями 600:1000 нейронов в первом слое с уменьшающимся количеством нейронов с шагом 100 в глубину НС, результаты модели усреднены по 3 тестам
*   модель 0:2 скрытыми слоями 

>лучшие результаты показали модели с 2 скрытыми слоями, в целом увеличение кол-ва нейронов приводит в увеличению точности, хотя наилучший рез-тат дала подель с 400 нейронов, что может быть случайным совпадением



*   модель с 3:5 скрытыми слоями

>в целом увеличение глубины сети и увеличение кол-ва нейронов в скрытых слоях приводит к увеличению точности, но точность глубокой модели меньше модели с меньшим количеством скрытых слоев 0:2 с одинаковым кол-вом нейронов

